# *SW12: use of pre-trained models and explainability (applied to "Sentiment Analysis" and "classification" tasks)*

inspired by [https://github.com/cdpierse/transformers-interpret#sequence-classification-explainer](https://github.com/cdpierse/transformers-interpret#sequence-classification-explainer), with input from this page: https://awesomeopensource.com/project/cdpierse/transformers-interpret

In [1]:
# !pip3 install -U transformers
# !pip3 install -U transformers_interpret

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
#USE distilbert-base-uncased-finetuned-sst-2-english: a distilbert model finetuned on a sentiment analysis task.

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
# With both the model and tokenizer initialized we are now able to get explanations on an example text.
from transformers_interpret import SequenceClassificationExplainer, ZeroShotClassificationExplainer

In [19]:
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
word_attributions = cls_explainer("I love you, I like you")
word_attributions

[('[CLS]', 0.0),
 ('i', 0.27785420350970025),
 ('love', 0.7792373444562729),
 ('you', 0.3856004761295062),
 (',', -0.017697541091358095),
 ('i', 0.1207189402855532),
 ('like', 0.19091115155373445),
 ('you', 0.3399487345372883),
 ('[SEP]', 0.0)]

In [ ]:
cls_explainer.predicted_class_name

'POSITIVE'

In [ ]:
m = 'facebook/bart-large-mnli'

tokenizer2 = AutoTokenizer.from_pretrained(m)

model2 = AutoModelForSequenceClassification.from_pretrained(m)

Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 26.0kB/s]
Downloading: 100%|██████████| 1.13k/1.13k [00:00<00:00, 1.15MB/s]
Downloading: 100%|██████████| 878k/878k [00:00<00:00, 1.57MB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 986kB/s] 
Downloading: 100%|██████████| 1.29M/1.29M [00:00<00:00, 2.06MB/s]
Downloading: 100%|██████████| 1.52G/1.52G [02:08<00:00, 12.6MB/s]


In [ ]:
zero_shot_explainer = ZeroShotClassificationExplainer(model2, tokenizer2)

In [ ]:
input2 = "I love you, I like you but sometimes I'm not sure I would like to marry you..."

In [ ]:
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
expl1 = cls_explainer(input2)

In [ ]:
expl1

[('[CLS]', 0.0),
 ('i', -0.17919146733536945),
 ('love', -0.10876658820352109),
 ('you', -0.1265794130538051),
 (',', -0.014920844993548758),
 ('i', -0.05214202142316559),
 ('like', 0.017265334274621937),
 ('you', -0.09352812008772914),
 ('but', 0.5639923594271705),
 ('sometimes', -0.3755296090841353),
 ('i', 0.046014115402454446),
 ("'", -0.15290425715631567),
 ('m', -0.1687553198408003),
 ('not', -0.3937190967701838),
 ('sure', 0.3696136745137524),
 ('i', 0.25223015334531246),
 ('would', 0.21676789155796447),
 ('like', 0.07449702447915288),
 ('to', -0.0017056287591338947),
 ('marry', 0.005898448957407097),
 ('you', 0.00805418736919376),
 ('.', 0.07557921845162473),
 ('.', -0.03429728829928755),
 ('.', 0.01368848883033041),
 ('[SEP]', 0.0)]

In [ ]:
expl2 = zero_shot_explainer(
    input2,
    labels = ["negative", "neutral", "positive"],
)

In [ ]:
zero_shot_explainer.predicted_label

'negative'

In [ ]:
nn = zero_shot_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
negative,negative (0.40),negative,1.31,"#s I love you , I like you but sometimes I 'm not sure I would like to marry you . . ."
neutral,neutral (0.39),neutral,1.48,"#s I love you , I like you but sometimes I 'm not sure I would like to marry you . . ."
positive,positive (0.21),positive,0.28,"#s I love you , I like you but sometimes I 'm not sure I would like to marry you . . ."


In [ ]:
expl2 = cls_explainer("I love you, I like you, I also kinda dislike you", class_name="NEGATIVE")

In [ ]:
cls_explainer.predicted_class_name

'POSITIVE'

In [ ]:
nn = cls_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,POSITIVE (0.00),NEGATIVE,-1.63,"[CLS] i love you , i like you , i also kinda dislike you [SEP]"


In [ ]:
expl2 = cls_explainer("I love you, I like you, I also kinda dislike you", class_name="POSITIVE")

In [ ]:
cls_explainer.predicted_class_name

'POSITIVE'

In [ ]:
nn = cls_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,POSITIVE (1.00),POSITIVE,1.63,"[CLS] i love you , i like you , i also kinda dislike you [SEP]"


In [ ]:
text2 = """The capitalization of the AMD chip maker is going to sink, due to the chip shortage and the need for huge investments for scaling up the production capacity. 
    At the same time, small local producer can became relevant player in a now under pressure market."""

In [ ]:
candidate_labels = ['music', 'energy', 'environment', 'dancing', 'exploration', 'aviation', "finance", "technology", "sports", "economy", "ecology", "stock"]

word_attributions = zero_shot_explainer(
    text2,
#    labels = ["finance", "technology", "sports", "economy", "ecology", "stock"],
    labels = candidate_labels,
)

In [ ]:
import numpy as np

#print(type(word_attributions))

limit=25
          
header = "{:<16}".format(" ")
line = list()
v = list()
for key, vals in zip(word_attributions.keys(), word_attributions.values()):
    header += "{:<13}".format(key)
    i = 0
    
    for val in vals[:limit]:
        if ((i+1) > len(line)):
            line.append("{:<15}".format(val[0]))
            v.append(list())   
        line[i] += "{:>6.3f}       ".format(val[1])
        v[i].append(val[1])
        i += 1
            
#print(v)            
   
header += "{:<13}".format("STD.DEV")    
    
print(header)
i = 0
for elem in line:
    std = np.std(v[i])
    elem += "{:>6.3f}       ".format(std)
    #avg = np.average(v[i])
    #elem += "{:>6.3f}       ".format(avg)
    print(elem)
    i += 1

                music        energy       environment  dancing      exploration  aviation     finance      technology   sports       economy      ecology      stock        STD.DEV      
<s>             0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000       
The             0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000       
capital        -0.022       -0.085        0.002       -0.007       -0.110        0.011       -0.095       -0.157       -0.064       -0.045        0.090        0.039        0.066       
ization        -0.120       -0.062       -0.001        0.016        0.119        0.025       -0.120        0.242       -0.070       -0.050        0.173       -0.043        0.110       
of             -0.010       -0.717        0.026        0.205       -0.191 

In [ ]:
zero_shot_explainer.predicted_label

'technology'

In [ ]:
nn = zero_shot_explainer.visualize();

In [ ]:
new_candidate_labels =  ['stock market', 'market', "finance", "technology", "economy", "stock"]

word_attributions = zero_shot_explainer(
    text2,
    labels = new_candidate_labels,
)

In [ ]:
limit=25
          
header = "{:<16}".format(" ")
line = list()
v = list()
for key, vals in zip(word_attributions.keys(), word_attributions.values()):
    header += "{:<13}".format(key)
    i = 0
    
    for val in vals[:limit]:
        if ((i+1) > len(line)):
            line.append("{:<15}".format(val[0]))
            v.append(list())   
        line[i] += "{:>6.3f}       ".format(val[1])
        v[i].append(val[1])
        i += 1
            
#print(v)            
   
header += "{:<13}".format("STD.DEV")    
    
print(header)
i = 0
for elem in line:
    std = np.std(v[i])
    elem += "{:>6.3f}       ".format(std)
    #avg = np.average(v[i])
    #elem += "{:>6.3f}       ".format(avg)
    print(elem)
    i += 1

                stock market market       finance      technology   economy      stock        STD.DEV      
<s>             0.000        0.000        0.000        0.000        0.000        0.000        0.000       
The             0.000        0.000        0.000        0.000        0.000        0.000        0.000       
capital        -0.095       -0.136       -0.095       -0.157       -0.045        0.039        0.064       
ization        -0.235        0.432       -0.120        0.242       -0.050       -0.043        0.228       
of             -0.954        0.441       -0.637       -0.463       -0.304        0.276        0.491       
the            -0.091        0.098       -0.051       -0.120       -0.181        0.096        0.105       
AMD             0.028       -0.079        0.056       -0.058       -0.207        0.017        0.088       
chip            0.035       -0.104       -0.018       -0.088        0.103       -0.334        0.138       
maker          -0.029        0.114  

In [ ]:
zero_shot_explainer.predicted_label

'market'

In [ ]:
nn = zero_shot_explainer.visualize();

In [ ]:
#input from https://www.swissinfo.ch/eng/how-sustainable-fuels-created-from-thin-air-could-solve-the-energy-crisis/47120934 only the headline (limitation of the input lenght)

text =""""
A refinery that produces fuels from sunlight and air… the scenario sounds like science fiction. 
Yet Swiss scientists claim to have done just that and built a potentially scalable way of producing sustainable fuels. 
A Swiss spin-off is working to commercialise the technology.
"""

In [ ]:
#import transformers
# print(transformers.__version__)

from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [ ]:
ret = classifier(text, candidate_labels, multi_label=True)

In [ ]:
for e in zip(ret['labels'],ret['scores']):
    #print(e[1])
    print("class {:>20}  - relevance {:>6.3f}".format(e[0],e[1]))

class           technology  - relevance  0.983
class               energy  - relevance  0.922
class          environment  - relevance  0.640
class              economy  - relevance  0.384
class          exploration  - relevance  0.081
class              finance  - relevance  0.079
class                stock  - relevance  0.041
class              dancing  - relevance  0.015
class             aviation  - relevance  0.011
class              ecology  - relevance  0.007
class                music  - relevance  0.002
class               sports  - relevance  0.001


In [ ]:
zero_shot_explainer(
    text,
    labels = candidate_labels
)

{'music': [('<s>', 0.0),
  ('"', 0.0),
  ('Ċ', 0.11088524846975557),
  ('A', 0.050180006123860935),
  ('refinery', 0.14624476560406088),
  ('that', -0.03754144899345838),
  ('produces', -0.277329819801346),
  ('fuels', 0.028148508917155396),
  ('from', 0.22892764819711572),
  ('sunlight', 0.13881053698846052),
  ('and', 0.37087259748874174),
  ('air', 0.2136406586542295),
  ('âĢ¦', -0.28363367626220304),
  ('the', 0.09778385705455998),
  ('scenario', 0.16551965992150827),
  ('sounds', 0.08264051362207814),
  ('like', 0.5343652437784276),
  ('science', -0.2079813238888459),
  ('fiction', 0.15836240842408447),
  ('.', -0.034786875424108896),
  ('Yet', 2.835328542106649e-05),
  ('Swiss', -0.02132723310032684),
  ('scientists', -0.03299952422319383),
  ('claim', 0.028969925781236385),
  ('to', -0.03593543006056325),
  ('have', 0.008767398325731588),
  ('done', 0.0006274314278981721),
  ('just', -0.026741335717444477),
  ('that', 0.07910175780983326),
  ('and', 0.01008791710560909),
  ('bui

In [ ]:
zero_shot_explainer.predicted_label

'technology'

In [ ]:
zero_shot_explainer.visualize();

In [ ]:
res = zero_shot_explainer(
    text,
    labels = ["positive","neutral","negative"]
)

In [ ]:
zero_shot_explainer.predicted_label

'negative'

In [ ]:
zero_shot_explainer.visualize();

In [ ]:

sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

Downloading: 100%|██████████| 687/687 [00:00<00:00, 687kB/s]
Downloading: 100%|██████████| 1.32G/1.32G [01:53<00:00, 12.6MB/s]
Downloading: 100%|██████████| 256/256 [00:00<00:00, 256kB/s]
Downloading: 100%|██████████| 780k/780k [00:00<00:00, 1.43MB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 986kB/s] 
Downloading: 100%|██████████| 150/150 [00:00<00:00, 150kB/s]


In [ ]:
sentiment_analysis(text)

[{'label': 'POSITIVE', 'score': 0.9976629018783569}]

In [ ]:
cls_explainer = SequenceClassificationExplainer(
    sentiment_analysis.model,
    sentiment_analysis.tokenizer)

In [ ]:
word_attributions = cls_explainer(text)

In [ ]:
word_attributions

[('<s>', 0.0),
 ('"', 0.007870082009781332),
 ('Ċ', 0.00693632912658196),
 ('A', 0.04512590395020376),
 ('refinery', -0.025060749305012717),
 ('that', 0.055025609127968694),
 ('produces', -0.1306211568197365),
 ('fuels', -0.1039348125765158),
 ('from', -0.08677638833888772),
 ('sunlight', -0.21546397136173756),
 ('and', -0.0333374107793791),
 ('air', 0.02556695937016207),
 ('âĢ¦', -0.029080639515637),
 ('the', 0.050151759016345096),
 ('scenario', -0.04618141606793187),
 ('sounds', 0.019277531403004932),
 ('like', -0.020047256361169864),
 ('science', -0.07883026156733099),
 ('fiction', -0.19068569860133877),
 ('.', -0.020211790784732335),
 ('Yet', -0.04456579133376124),
 ('Swiss', -0.05453737695036269),
 ('scientists', 0.02649501347160408),
 ('claim', -0.023967887152561713),
 ('to', 0.03217060515337784),
 ('have', -0.0007655103731317917),
 ('done', 0.0003567744316199803),
 ('just', 0.037033832204685445),
 ('that', 0.02936884327259373),
 ('and', 0.03433691147603564),
 ('built', 0.0360684

In [ ]:
cls_explainer.predicted_class_name

'POSITIVE'

In [ ]:
cls_explainer.visualize();

## EXERCISE 1

use the review dataset, as from the folder in the data, and consider scores 1/2 as negative, 3 as neutral and 4/5 as positive

loop on all then entry and compute the class. Based on the real class, create a confusion matrix: how it performs?

In [28]:
import pandas as pd
import csv
#reviews = pd.read_csv("data/reviews.csv", sep ='',)
listOfReviews = []
i = 0
with open('data/reviews.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if i < 20:
            expl2 = cls_explainer(row["content"], class_name="POSITIVE")
            nn = cls_explainer.visualize()
            listOfReviews.append(row["content"])
            i = i+1

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 6206: character maps to <undefined>

In [5]:
print(reviews)
reviewsexpl2 = cls_explainer("I love you, I like you, I also kinda dislike you", class_name="NEGATIVE")

                userName                                          userImage  \
0          Andrew Thomas  https://lh3.googleusercontent.com/a-/AOh14GiHd...   
1           Craig Haines  https://lh3.googleusercontent.com/-hoe0kwSJgPQ...   
2          steven adkins  https://lh3.googleusercontent.com/a-/AOh14GiXw...   
3       Lars Panzerbjørn  https://lh3.googleusercontent.com/a-/AOh14Gg-h...   
4          Scott Prewitt  https://lh3.googleusercontent.com/-K-X1-YsVd6U...   
...                  ...                                                ...   
15741          Tammy Kay  https://lh3.googleusercontent.com/a-/AOh14GhYP...   
15742          Ysm Johan  https://lh3.googleusercontent.com/a-/AOh14Ggmd...   
15743      casey dearden  https://lh3.googleusercontent.com/a-/AOh14Gg2U...   
15744     Jerry G Tamate  https://lh3.googleusercontent.com/a-/AOh14GiTP...   
15745  Ahmed elsalamouni  https://lh3.googleusercontent.com/-9QSxVUhCoDI...   

                                                 co

NameError: name 'cls_explainer' is not defined

### Bonus: explanation on Q&A

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers_interpret import QuestionAnsweringExplainer

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

qa_explainer = QuestionAnsweringExplainer(
    model,
    tokenizer,
)

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 14.0kB/s]
Downloading: 100%|██████████| 443/443 [00:00<00:00, 443kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 832kB/s] 
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 1.01MB/s]
Downloading: 100%|██████████| 1.25G/1.25G [01:46<00:00, 12.6MB/s]


In [ ]:
context = """
In Artificial Intelligence and machine learning, Natural Language Processing relates to the usage of machines to process and understand human language.
Many researchers currently work in this space.
"""

word_attributions = qa_explainer(
    "What is natural language processing ?",
    context,
)

In [ ]:
word_attributions

{'start': [('[CLS]', 0.0),
  ('what', 0.917717061867351),
  ('is', 0.1338213861414588),
  ('natural', 0.08061731698784134),
  ('language', 0.013138230044765855),
  ('processing', 0.11135945915876394),
  ('?', 0.008580120143897719),
  ('[SEP]', -0.09646334048536524),
  ('in', 0.01545606320896786),
  ('artificial', 0.04720979393539745),
  ('intelligence', 0.02668683069056011),
  ('and', 0.016753607049298964),
  ('machine', -0.08429576178208778),
  ('learning', 0.004482511600234183),
  (',', -0.02401029530321712),
  ('natural', -0.0016759040611807657),
  ('language', 0.002681433280950527),
  ('processing', 0.06773119884848856),
  ('relates', 0.03884567065125608),
  ('to', 0.009783733105059877),
  ('the', -0.026650828552455926),
  ('usage', -0.010675208218277504),
  ('of', 0.015346547746357412),
  ('machines', -0.08277993141643336),
  ('to', 0.12861371511135003),
  ('process', 0.19540094680181358),
  ('and', 0.009943096833436344),
  ('understand', 0.0068367995968998665),
  ('human', 0.0502

In [ ]:
qa_explainer.predicted_answer

'usage of machines to process and understand human language'

In [ ]:
nn = qa_explainer.visualize()

In [ ]:
context = """
The Old Man and the Sea, short heroic novel by Ernest Hemingway, published in 1952 and awarded the 1953 
Pulitzer Prize for fiction. It was his last major work of fiction. The story centres on an aging fisherman 
who engages in an epic battle to catch a giant marlin.
"""

In [ ]:
word_attributions = qa_explainer(
    "What is the story called?",
    context,
)

print (qa_explainer.predicted_answer)
nn = qa_explainer.visualize()

the old man and the sea


In [ ]:
word_attributions = qa_explainer(
    "What is the story about?",
    context,
)

print (qa_explainer.predicted_answer)
nn = qa_explainer.visualize()

an aging fisherman


In [ ]:
word_attributions = qa_explainer(
    "which prize has the novel received?",
    context,
)

print (qa_explainer.predicted_answer)
nn = qa_explainer.visualize()

pulitzer prize for fiction


In [ ]:
word_attributions = qa_explainer(
    "Why is the fish in the novel?",
    context,
)

print (qa_explainer.predicted_answer)
nn = qa_explainer.visualize()

to catch a giant mar ##lin
